Retention – один из самых важных показателей в компании. Ваша задача – написать функцию, которая будет считать retention игроков (по дням от даты регистрации игрока). Данные лежат в папке shared и имеют следующую структуру:

shared/problem1-reg_data.csv – данные о времени регистрации<br>
shared/problem1-auth_data.csv – данные о времени захода пользователей в игру

Функция должна быть написана на python. В ходе решения можно тестировать работу функции как на полном датасете, так и на части (сэмпле) данных.

## Импорт библиотек

In [1]:
import pandas as pd
import datetime
import numpy as np


## Загрузка данных
reg - данные о времени регистрации<br>
work - данные о времени захода пользователей в игру 

In [2]:
reg = pd.read_csv('/shared/problem1-reg_data.csv',\
                  sep = ';')
work = pd.read_csv('/shared/problem1-auth_data.csv',\
                   sep = ';')

## Анализ данных

Для начала посмотрим структуру данных, определим есть ли пропуски в таблицах,повторяются ли uid, посмотрим на тип данных

#### Оценим данные о времени регистрации, датафрейм reg

In [3]:
reg.head()

,reg_ts,uid
0,911382223,1
1,932683089,2
2,947802447,3
3,959523541,4
4,969103313,5


In [4]:
reg.info()

<class 'pandas.core.frame.DataFrame'>
,RangeIndex: 1000000 entries, 0 to 999999
,Data columns (total 2 columns):
, #   Column  Non-Null Count    Dtype
,---  ------  --------------    -----
, 0   reg_ts  1000000 non-null  int64
, 1   uid     1000000 non-null  int64
,dtypes: int64(2)
,memory usage: 15.3 MB


In [5]:
reg.uid.nunique()

1000000

In [6]:
reg.isnull().sum()

reg_ts    0
uid       0
dtype: int64

<br>Данные без пропусков,без повторов uid пользователей,тип данных int64, размер датафрейма 1000000. Имеем 1000000 пользователей с уникальным uid

#### Оценим данные о времени захода пользователей в игру, датафрейм reg

In [7]:
work.head()

,auth_ts,uid
0,911382223,1
1,932683089,2
2,932921206,2
3,933393015,2
4,933875379,2


In [8]:
work.info()

<class 'pandas.core.frame.DataFrame'>
,RangeIndex: 9601013 entries, 0 to 9601012
,Data columns (total 2 columns):
, #   Column   Dtype
,---  ------   -----
, 0   auth_ts  int64
, 1   uid      int64
,dtypes: int64(2)
,memory usage: 146.5 MB


In [9]:
work.isnull().sum()

auth_ts    0
uid        0
dtype: int64

In [10]:
work.uid.nunique()

1000000

In [11]:
work.uid.value_counts().head()

2      1929
108    1397
158    1367
176    1331
191    1296
Name: uid, dtype: int64

Данные без пропусков, тип данных int64, колличество строк 9601013. Уникальных uid 1000000, многие пользователи заходили в игру несколько раз

#### Объединие данных
Объединим данные двух датафреймов на основе датафрейма work 

In [12]:
all_info = pd.merge(work,reg,on = 'uid')[['uid','reg_ts','auth_ts']]

In [13]:
all_info.head()

,uid,reg_ts,auth_ts
0,1,911382223,911382223
1,2,932683089,932683089
2,2,932683089,932921206
3,2,932683089,933393015
4,2,932683089,933875379


In [14]:
all_info.info()

<class 'pandas.core.frame.DataFrame'>
,Int64Index: 9601013 entries, 0 to 9601012
,Data columns (total 3 columns):
, #   Column   Dtype
,---  ------   -----
, 0   uid      int64
, 1   reg_ts   int64
, 2   auth_ts  int64
,dtypes: int64(3)
,memory usage: 293.0 MB


In [15]:
all_info.isnull().sum()

uid        0
reg_ts     0
auth_ts    0
dtype: int64

Количество строк совпадает с датафреймом work, данные не потеряли, пустых значений нет

#### Функция для retention
Теперь напишем функцию для retention: cohort(df_all_info,start,stop).<br>
На вход она будет принимать 3 параметра:<br>
 - df_all_info - датафрейм, в котором нужно найти retention. Датафрейм должен содержать столбцы reg_ts,auth_ts,uid в формате int64. Для времени регистрации и времени захода в игру используется unix - время<br>
 - start - дата,с которой мы хотим посмотреть retention, формат str, вид '2020-08-23'<br>
 - stop - дата,до которой мы хотим посмотреть retention, формат str, вид '2020-09-23' 
Выводить будет retention в % за заданный период
 

In [21]:
def cohort(df_all_info,start,stop):
    #скопируем датафрейм,чтобы не вносить изменения в исходный
    df = df_all_info.copy()
    #проверка на правильность данных по столбцам 
    col = True
    for i in df.columns:
        if i not in ['uid', 'reg_ts', 'auth_ts']:  
            col = False
            break
        continue 
    
    if len(df.columns) < 3 or col == False:
            print('Проверьте исходный датафрйем. Датафрейм должен содержать 3 столбца: uid, reg_ts и auth_ts')
    else:
        #переводим даты из int64 в datetime64
        df.reg_ts = pd.to_datetime(df.reg_ts,unit = 's').dt.date
        df.auth_ts = pd.to_datetime(df.auth_ts,unit = 's').dt.date
        df.reg_ts = pd.to_datetime(df.reg_ts)
        df.auth_ts = pd.to_datetime(df.auth_ts)
        #проверка правильности данных по дате
        if  (df.reg_ts.min() <= pd.to_datetime(start) <= df.reg_ts.max() and\
            df.reg_ts.min() <= pd.to_datetime(stop) <= df.reg_ts.max()) and\
            start <= stop :
                #обрезаем исходный датафрейм под наши даты
                df_rezult = df.query('@start<=reg_ts <= @stop and @start <=auth_ts <= @stop').copy()
                #расчитываем сколько прошло дней после регистрации 
                df_rezult['days'] = (df_rezult.auth_ts - df_rezult.reg_ts)/np.timedelta64(1, 'D')
                #формируем когорты
                cohort_result = df_rezult.groupby(['days','reg_ts'])\
                                         .agg(users =('uid','nunique'))\
                                         .reset_index()\
                                         .pivot_table(index = 'reg_ts', columns = 'days', values = 'users')
                #переводим в проценты
                retention_matrix = cohort_result.divide(cohort_result[0], axis = 0).round(4)*100
                return retention_matrix
           
        else: 
            print(f'Проверьте правильность написания даты. Дата должна быть формата "2020-09-23" и лежать в диапазоне' +
            f' от "{str(df.reg_ts.min().date())}" до "{str(df.reg_ts.max().date())}".')    

In [22]:
#проверка дат, start > stop
cohort(all_info,'2020-08-27','2020-08-25')

Проверьте правильность написания даты. Дата должна быть формата "2020-09-23" и лежать в диапазоне от "1998-11-18" до "2020-09-23".


In [23]:
#проверка дат,выход из диапазона
cohort(all_info,'1997-08-25','2019-08-27')

Проверьте правильность написания даты. Дата должна быть формата "2020-09-23" и лежать в диапазоне от "1998-11-18" до "2020-09-23".


In [24]:
#проверка столбцов
all_info_test = all_info[['uid','reg_ts']].copy()
all_info_test

,uid,reg_ts
0,1,911382223
1,2,932683089
2,2,932683089
3,2,932683089
4,2,932683089
...,...,...
9601008,1110618,1600874034
9601009,1110619,1600874086
9601010,1110620,1600874139
9601011,1110621,1600874191


In [25]:
cohort(all_info_test,'1997-08-25','2019-08-27')

Проверьте исходный датафрйем. Датафрейм должен содержать 3 столбца: uid, reg_ts и auth_ts


In [26]:
#вывод retention за последний месяц в %
cohort(all_info,'2020-08-23','2020-09-23')

days,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0,30.0,31.0
reg_ts,,,,,,,,,,,,,,,,,,,,,
2020-08-23,100.0,2.50,3.84,4.87,4.93,6.98,8.19,5.70,5.12,4.55,...,4.42,3.39,3.91,3.14,3.65,4.16,2.88,3.07,3.39,1.79
2020-08-24,100.0,1.79,4.15,4.28,5.11,5.75,7.60,6.01,4.92,5.18,...,3.64,3.45,4.47,3.13,3.26,3.13,3.13,2.81,1.28,NaN
2020-08-25,100.0,1.98,4.40,4.79,5.68,5.30,5.30,6.25,3.70,5.74,...,3.57,3.25,3.25,4.40,2.81,3.13,3.19,2.30,NaN,NaN
2020-08-26,100.0,1.46,3.44,4.20,4.52,6.43,6.56,5.99,4.59,4.33,...,4.27,3.25,2.74,2.93,2.80,2.74,1.66,NaN,NaN,NaN
2020-08-27,100.0,1.65,3.37,4.77,5.21,5.53,6.17,5.72,4.32,5.40,...,2.99,2.99,3.88,3.37,2.73,1.46,NaN,NaN,NaN,NaN
2020-08-28,100.0,2.03,4.57,5.27,4.76,6.29,6.22,5.46,5.59,5.27,...,4.70,4.06,2.86,3.68,2.54,NaN,NaN,NaN,NaN,NaN
2020-08-29,100.0,2.53,3.74,4.63,5.70,5.51,6.97,5.70,4.56,6.02,...,4.25,3.49,3.55,1.71,NaN,NaN,NaN,NaN,NaN,NaN
2020-08-30,100.0,1.84,3.48,3.92,5.38,5.95,6.58,5.57,4.30,4.30,...,4.05,3.73,1.77,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-08-31,100.0,1.83,3.66,4.23,5.12,6.89,6.51,6.70,4.80,5.37,...,2.78,1.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
